# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_MNIST import MNISTDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [3]:
l2_lambda_regularization = 0.2
l1_lambda_regularization = 0.10
def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc3'))
    model.add(Dense(10, activation='softmax',kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [4]:

selected_features = 784


window_size = 1
window_stride = 1

dHandler_cmaps = MNISTDataHandler(selected_features, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

init


# Create Tunable Model

In [4]:
# min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
# #min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
# num_features = len(selected_features)

shape = 784
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 1
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 1

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
#tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data window_size of 1, stride of 1. Cros-Validation ratio 0
Printing shapes

Training data (X, y)
(60000, 784)
(60000, 10)
Testing data (X, y)
(10000, 784)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Testing data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = (selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = nFeatures*tModel.data_handler.sequence_length


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

# now = datetime.now()
# tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)

tModel.epochs = 200
tModel.train_model(learningRate_scheduler=lrate, verbose=1)




training without cv
Epoch 1/200
60000/60000 [==============================] - 2s 30us/step - loss: 31.8498 - mean_squared_error: 0.0900

In [11]:
# tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
# print("scores")

# cScores = tModel.scores
# #rmse = math.sqrt(cScores['score_1'])
# rmse2 = cScores['rmse']
# rhs = cScores['rhs']
# time = tModel.train_time

# i = range(1,len(tModel.y_test)+1)


# for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
#     print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
#                                                                     rul_prediction_rounded, 
#                                                                     true_rul))

# print(cScores)
# #print("RMSE: {}".format(rmse))
# print("RMSE2: {}".format(rmse2))
# print("RHS: {}".format(rhs))
# print("Time : {} seconds".format(time))

30/30 [==============================] - 0s 66us/step
scores
Engine 1, Predicted RUL 18.658912658691406, Rounded RUL 18.0, Real RUL 112.0
Engine 2, Predicted RUL 111.13060760498047, Rounded RUL 111.0, Real RUL 98.0
Engine 3, Predicted RUL 77.54817962646484, Rounded RUL 77.0, Real RUL 69.0
Engine 4, Predicted RUL 82.61211395263672, Rounded RUL 82.0, Real RUL 82.0
Engine 5, Predicted RUL 118.38198852539062, Rounded RUL 118.0, Real RUL 91.0
Engine 6, Predicted RUL 100.8348617553711, Rounded RUL 100.0, Real RUL 93.0
Engine 7, Predicted RUL 11.20897388458252, Rounded RUL 11.0, Real RUL 91.0
Engine 8, Predicted RUL 92.08351135253906, Rounded RUL 92.0, Real RUL 95.0
Engine 9, Predicted RUL 67.8717269897461, Rounded RUL 67.0, Real RUL 111.0
Engine 10, Predicted RUL 117.28495025634766, Rounded RUL 117.0, Real RUL 96.0
Engine 11, Predicted RUL 110.51206970214844, Rounded RUL 110.0, Real RUL 97.0
Engine 12, Predicted RUL 109.71183013916016, Rounded RUL 109.0, Real RUL 124.0
Engine 13, Predicted R

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

10000/10000 [==============================] - 1s 60us/step
scores
Engine 1, Predicted RUL 0.09875352680683136, Rounded RUL 0.0, Real RUL 0.0
Engine 2, Predicted RUL 0.11234200745820999, Rounded RUL 0.0, Real RUL 0.0
Engine 3, Predicted RUL 0.09923621267080307, Rounded RUL 0.0, Real RUL 0.0
Engine 4, Predicted RUL 0.10217634588479996, Rounded RUL 0.0, Real RUL 0.0
Engine 5, Predicted RUL 0.09738582372665405, Rounded RUL 0.0, Real RUL 0.0
Engine 6, Predicted R